<a href="https://colab.research.google.com/github/stephanedenis/PaniniFS-Research/blob/main/notebooks/colab_dhatu_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Dhātu Analysis - Simple & Fast

Carnet Colab simple pour analyser les dhātus et interagir avec le système de collecte.

In [ ]:
# 📦 Setup rapide
import os, json, requests, time
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt

# GitHub setup
REPO_URL = "https://github.com/stephanedenis/PaniniFS-Research"
if not os.path.exists('PaniniFS-Research'):
    !git clone $REPO_URL
    os.chdir('PaniniFS-Research')
else:
    os.chdir('PaniniFS-Research')
    !git pull origin main

print("✅ Repository ready!")

In [ ]:
# 🔍 Analyseur Dhātu Simple
class SimpleDhatuAnalyzer:
    def __init__(self):
        self.dhatu_patterns = {
            'भू': r'भू|bhū|être|being',
            'कृ': r'कृ|kṛ|faire|doing|make',
            'गम्': r'गम्|gam|aller|going|move',
            'दा': r'दा|dā|donner|giving|give',
            'स्था': r'स्था|sthā|être debout|standing'
        }
        self.results = []
    
    def analyze_text(self, text, source="unknown"):
        """Analyse rapide d'un texte"""
        matches = {}
        for dhatu, pattern in self.dhatu_patterns.items():
            import re
            count = len(re.findall(pattern, text, re.IGNORECASE))
            if count > 0:
                matches[dhatu] = count
        
        result = {
            'source': source,
            'text_length': len(text),
            'dhatu_matches': matches,
            'total_matches': sum(matches.values()),
            'timestamp': datetime.now().isoformat()
        }
        self.results.append(result)
        return result
    
    def get_summary(self):
        """Résumé des analyses"""
        if not self.results:
            return "Aucune analyse effectuée"
        
        total_docs = len(self.results)
        total_matches = sum(r['total_matches'] for r in self.results)
        
        return f"📊 {total_docs} documents analysés, {total_matches} dhātus détectés"

# Initialiser l'analyseur
analyzer = SimpleDhatuAnalyzer()
print("🔍 Analyseur Dhātu initialisé!")

In [ ]:
# 📁 Charger les données collectées
def load_collected_data():
    """Charge les données du collecteur"""
    data_files = []
    
    # Chercher dans data/incremental_corpus/
    corpus_dir = 'data/incremental_corpus'
    if os.path.exists(corpus_dir):
        for file in os.listdir(corpus_dir):
            if file.endswith('.json') and 'batch_' in file:
                data_files.append(os.path.join(corpus_dir, file))
    
    print(f"📁 {len(data_files)} fichiers de données trouvés")
    return data_files

def analyze_collected_docs():
    """Analyse tous les documents collectés"""
    data_files = load_collected_data()
    
    for file_path in data_files[:5]:  # Limite à 5 fichiers pour commencer
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            if 'documents' in data:
                for doc in data['documents'][:10]:  # 10 docs par fichier
                    content = doc.get('content', '') + ' ' + doc.get('title', '')
                    result = analyzer.analyze_text(content, doc.get('source', 'unknown'))
                    print(f"📄 {doc.get('title', 'Sans titre')[:50]}... - {result['total_matches']} dhātus")
                    
        except Exception as e:
            print(f"❌ Erreur avec {file_path}: {e}")
    
    print(f"\n{analyzer.get_summary()}")

# Analyser les données
analyze_collected_docs()

In [ ]:
# 📊 Statistiques rapides
def show_quick_stats():
    """Affiche des stats rapides"""
    if not analyzer.results:
        print("❌ Aucune donnée à analyser")
        return
    
    # Créer DataFrame
    df_data = []
    for result in analyzer.results:
        for dhatu, count in result['dhatu_matches'].items():
            df_data.append({
                'dhatu': dhatu,
                'count': count,
                'source': result['source']
            })
    
    if df_data:
        df = pd.DataFrame(df_data)
        
        # Top dhātus
        top_dhatus = df.groupby('dhatu')['count'].sum().sort_values(ascending=False)
        print("🏆 Top Dhātus détectés:")
        for dhatu, count in top_dhatus.head().items():
            print(f"  {dhatu}: {count} occurrences")
        
        # Graphique simple
        plt.figure(figsize=(10, 6))
        top_dhatus.head().plot(kind='bar')
        plt.title('Distribution des Dhātus')
        plt.ylabel('Occurrences')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
        
        return df
    
    print("❌ Pas de données dhātu trouvées")

# Afficher les stats
df_results = show_quick_stats()

In [ ]:
# 🤖 Interaction avec le collecteur
def send_feedback_to_collector(feedback_data):
    """Envoie du feedback au collecteur"""
    feedback_file = 'colab_results/colab_feedback.json'
    
    # Créer le dossier si nécessaire
    os.makedirs('colab_results', exist_ok=True)
    
    # Sauvegarder le feedback
    with open(feedback_file, 'w', encoding='utf-8') as f:
        json.dump(feedback_data, f, ensure_ascii=False, indent=2)
    
    print(f"💾 Feedback sauvé dans {feedback_file}")
    
    # Tentative de commit vers GitHub (si configuré)
    try:
        # Vérifier si Git est configuré
        import subprocess
        result = subprocess.run(['git', 'config', 'user.email'], capture_output=True, text=True)
        
        if result.returncode == 0 and result.stdout.strip():
            # Git configuré, on peut commiter
            !git add colab_results/colab_feedback.json
            !git commit -m "🔄 Feedback Colab: optimisations collecteur"
            !git push origin main
            print("🚀 Feedback envoyé sur GitHub!")
        else:
            # Git pas configuré, juste sauvegarder localement
            print("💾 Feedback sauvé localement (Git non configuré)")
            print("ℹ️ Pour activer GitHub, configurez:")
            print("   !git config user.email 'your-email@example.com'")
            print("   !git config user.name 'Your Name'")
            
    except Exception as e:
        print(f"💾 Feedback sauvé localement: {e}")
        print("ℹ️ Synchronisation GitHub non disponible")

def create_feedback():
    """Crée un feedback basé sur l'analyse"""
    if not analyzer.results:
        return None
    
    # Calculer des métriques
    total_docs = len(analyzer.results)
    total_matches = sum(r['total_matches'] for r in analyzer.results)
    avg_matches = total_matches / total_docs if total_docs > 0 else 0
    
    feedback = {
        'timestamp': datetime.now().isoformat(),
        'analysis_summary': {
            'documents_analyzed': total_docs,
            'total_dhatu_matches': total_matches,
            'average_matches_per_doc': round(avg_matches, 2)
        },
        'collector_recommendations': {
            'focus_on_high_quality': avg_matches > 2,
            'increase_sanskrit_content': True,
            'preferred_sources': ['wikipedia_sanskrit', 'academic_papers']
        },
        'performance': {
            'processing_speed': 'fast',
            'gpu_usage': 'optimal',
            'next_batch_ready': True
        }
    }
    
    return feedback

# Créer et envoyer feedback
feedback = create_feedback()
if feedback:
    send_feedback_to_collector(feedback)
    print("✅ Feedback créé et envoyé!")
else:
    print("❌ Pas assez de données pour créer un feedback")

In [ ]:
# 🔄 Mode continu (optionnel)
def run_continuous_analysis(duration_minutes=10):
    """Mode d'analyse continu"""
    print(f"🔄 Démarrage analyse continue pour {duration_minutes} minutes...")
    
    start_time = time.time()
    last_analysis = 0
    
    while (time.time() - start_time) < (duration_minutes * 60):
        # Vérifier s'il y a de nouvelles données toutes les 30 secondes
        if time.time() - last_analysis > 30:
            !git pull origin main --quiet
            
            # Analyser nouvelles données
            data_files = load_collected_data()
            if data_files:
                print(f"📊 {len(data_files)} fichiers trouvés, analyse en cours...")
                analyze_collected_docs()
                
                # Envoyer feedback
                feedback = create_feedback()
                if feedback:
                    send_feedback_to_collector(feedback)
            
            last_analysis = time.time()
        
        time.sleep(5)  # Attendre 5 secondes
    
    print("✅ Analyse continue terminée!")

# Décommenter pour lancer le mode continu
# run_continuous_analysis(5)  # 5 minutes

## 🎯 Instructions d'utilisation

1. **Exécutez les cellules dans l'ordre**
2. **Analysez les données collectées** avec la cellule "data_loader"
3. **Visualisez les stats** avec "quick_stats"
4. **Envoyez du feedback** avec "collector_interaction"
5. **Mode continu optionnel** pour surveillance longue durée

Le carnet synchronise automatiquement avec GitHub et envoie des recommendations au collecteur local.